In [1]:



import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device





device(type='cuda')

In [2]:


name = "google/flan-t5-xl"

path = f'./data/{name}/base_model'


# model = AutoModelForCausalLM.from_pretrained(name,device_map= 'auto',
#                                              cache_dir = path )


# tokenizer = AutoTokenizer.from_pretrained(name,  cache_dir = path)



tokenizer = AutoTokenizer.from_pretrained(name, cache_dir = path)
model = AutoModelForSeq2SeqLM.from_pretrained(name ,cache_dir = path,  device_map="auto" )



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
input_text = "translate English to french: hello "

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))
input_ids

tensor([[13959,  1566,    12, 20609,    10, 21820,     3,     1]],
       device='cuda:0')

In [30]:
import math

input_text = "translate English to french: The sun peeked over the horizon, casting a warm glow across the sleepy town. Birds chirped merrily, greeting the new day with their cheerful songs. Wisps of clouds painted the sky in shades of pink and orange, a canvas of natural beauty. People emerged from their homes, ready to embrace the opportunities that awaited them. "

# Split the input text into chunks for decoding
max_sequence_length = 10  # Define the maximum sequence length for each chunk

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

input_ids_chunks = [input_ids[0,i:i+max_sequence_length].unsqueeze(dim=0) for i in range(0, len(input_text), max_sequence_length)]

print(input_ids_chunks)
# Initialize an empty list to store the decoded chunks
decoded_chunks = []

# Decode each chunk
for ids_chunk in input_ids_chunks:
    outputs = model.generate(ids_chunk)
    decoded_chunk = tokenizer.decode(outputs[0])
    print(decoded_chunk)
    decoded_chunks.append(decoded_chunk)




[tensor([[13959,  1566,    12, 20609,    10,    37,  1997, 20584,    15,    26]],
       device='cuda:0'), tensor([[  147,     8,     3, 12158,     6, 16050,     3,     9,  1978, 16507]],
       device='cuda:0'), tensor([[  640,     8,  2085,    63,  1511,     5,  9992,     7,     3, 11727]],
       device='cuda:0'), tensor([[ 3138,     3,    51, 21301,   120,     6, 18660,     8,   126,   239]],
       device='cuda:0'), tensor([[   28,    70, 25675,  3605,     5,  2142,     7,   102,     7,    13]],
       device='cuda:0'), tensor([[15328,  7445,     8,  5796,    16, 11215,    13,  5571,    11,  5470]],
       device='cuda:0'), tensor([[    6,     3,     9, 10788,    13,   793,  2790,     5,  2449, 13999]],
       device='cuda:0'), tensor([[  45,   70, 2503,    6, 1065,   12, 9599,    8, 1645,   24]],
       device='cuda:0'), tensor([[    3,     9, 13106,   135,     5,     3,     1]], device='cuda:0'), tensor([], device='cuda:0', size=(1, 0), dtype=torch.int64), tensor([], device='cud

RuntimeError: cannot reshape tensor of 0 elements into shape [-1, 0] because the unspecified dimension size -1 can be any value and is ambiguous

In [21]:
input_ids

tensor([[13959,  1566,    12, 20609,    10,    37,  1997, 20584,    15,    26,
           147,     8,     3, 12158,     6, 16050,     3,     9,  1978, 16507,
           640,     8,  2085,    63,  1511,     5,  9992,     7,     3, 11727,
          3138,     3,    51, 21301,   120,     6, 18660,     8,   126,   239,
            28,    70, 25675,  3605,     5,  2142,     7,   102,     7,    13,
         15328,  7445,     8,  5796,    16, 11215,    13,  5571,    11,  5470,
             6,     3,     9, 10788,    13,   793,  2790,     5,  2449, 13999,
            45,    70,  2503,     6,  1065,    12,  9599,     8,  1645,    24,
             3,     9, 13106,   135,     5,     3,     1]], device='cuda:0')

In [22]:
import time

input_text = "translate English to french: The sun peeked over the horizon, casting a warm glow across the sleepy town. Birds chirped merrily, greeting the new day with their cheerful songs. Wisps of clouds painted the sky in shades of pink and orange, a canvas of natural beauty. People emerged from their homes, ready to embrace the opportunities that awaited them. "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)

generated_ids = outputs[0]
generated_text = ""

for token_id in generated_ids:
    token = tokenizer.decode(token_id)
    generated_text = token + ' '
    print(generated_text, end="", flush=True)
    time.sleep(0.05)  # Adjust the delay time as needed
print()  # Print newline after completing the output


c:\Users\Code212CU3\Desktop\adilo\llama2\venv\Lib\site-packages\transformers\generation\utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Le soleil  s ' est  é lev é sur  l ' horizon ,  e t 


In [5]:
# def get_llama2_reponse(prompt, max_new_tokens=50):
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)
#     outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return response




In [6]:
# prompt = "Q:si je suis né en 1996 c quoi mon age en 2024? A:"
# get_llama2_reponse(prompt, max_new_tokens=50)